#### Set up Agent primitives

In [ ]:
## 1. Model
from strands.models import BedrockModel
bedrock_model = BedrockModel(model_id="us.amazon.nova-pro-v1:0")

## 2. System Prompt / Instructions
system_prompt = f"You are a helpful assistant that provides concise answers. "

## 3. Conversation Manager
from strands.agent.conversation_manager import SlidingWindowConversationManager
conv_manager = SlidingWindowConversationManager(window_size=4)

#### Use in built in tools registry

In [ ]:
## Tools
from strands_tools import calculator, current_time
tool_list = [calculator, current_time]

#### Create the Agent

In [ ]:
## Create Agent
from strands import Agent
agent = Agent(model=bedrock_model,
              tools = tool_list,
              system_prompt=system_prompt, 
              conversation_manager=conv_manager
            )

#### Invoke the Agent

In [ ]:
## Invoke Agent
response = agent("What is the current time")

In [ ]:
print(f"Total tokens: {response.message}")
print(f"Total tokens: {response.metrics.accumulated_usage}")
print(f"Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
print(f"Tools used: {list(response.metrics.tool_metrics.keys())}")

In [ ]:
response = agent("What is 123 × 456?")

In [ ]:
print(f"Total tokens: {response.message}")
print(f"Total tokens: {response.metrics.accumulated_usage}")
print(f"Execution time: {sum(response.metrics.cycle_durations):.2f} seconds")
print(f"Tools used: {list(response.metrics.tool_metrics.keys())}")

#### Conversation Memory

In [ ]:
for msg in agent.messages:
    print(msg)

#### Logging

In [ ]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

In [ ]:
## Invoke Agent
response = agent("What is the 10*200+500")

#### Adding Guardrail to the Agent

In [ ]:
guardrail_id = "0a2rrw070xhi"
guardail_version = "1"

In [ ]:
from strands.models import BedrockModel
model_with_guards = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    guardrail_id=guardrail_id,              # Your Bedrock guardrail ID
    guardrail_version=guardail_version,     # Guardrail version
    guardrail_trace="enabled",              # Enable trace info for debugging
)

In [ ]:
## Create Agent
from strands import Agent
agent_with_guards = Agent(model=model_with_guards,
              tools = tool_list,
              system_prompt=system_prompt, 
              conversation_manager=conv_manager
            )

In [ ]:
response = agent_with_guards("who is donald trump")

In [ ]:
if response.stop_reason == "guardrail_intervened":
    print("Content was blocked by guardrails, conversation context overwritten!")